# To access the Mongo_db follow instructions:

To dump your mongo_db call this command on your terminal

 - mongodump --db database_name --collection collection_name (It will create a folder called dump in the folder active directory)

To import mongo_db file to mongodb you can use the following command on your terminal:

 - mongorestore --db database_name path_to_bson_file

In [8]:
# Database dependencies.
from sqlalchemy import create_engine

#Sqlite engine
engine = create_engine("sqlite:///USEnergy.sqlite", echo=True)
conn = engine.connect()
 
# Api dependencies
import requests 

#Json dependencies
from pprint import pprint

#API key
from DriversConfig.config import API_KEY #Save API key as API_KEY in a folder called DriversConfig

#DataFrame dependencies
import pandas as pd 

#Webscrapping 
from bs4 import BeautifulSoup as bs
from selenium import webdriver

#Random crap
import time

#Settings for headless mode.
options = webdriver.ChromeOptions()
options.add_argument('headless')

#path to the driver and load the options.
browser = webdriver.Chrome("/Users/Sebastian/Documents/GitHub/Data Visualization Bootcamp/Sebastian Homework/Group Project2 - ProjectEnergy/Sebastian/DriversConfig/chromedriver",chrome_options = options) #Dowload Selenium chromedriver and save the driver in the folder in the repo and update Absolute path.





2020-09-04 20:24:29,644 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-09-04 20:24:29,645 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:24:29,648 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-09-04 20:24:29,649 INFO sqlalchemy.engine.base.Engine ()


In [2]:
#Lower 48 list.
states = ["AL", "AK", "AZ", "AR", "CA", "CO", "CT", "DC", "DE", "FL", "GA", 
          "HI", "ID", "IL", "IN", "IA", "KS", "KY", "LA", "ME", "MD", 
          "MA", "MI", "MN", "MS", "MO", "MT", "NE", "NV", "NH", "NJ", 
          "NM", "NY", "NC", "ND", "OH", "OK", "OR", "PA", "RI", "SC", 
          "SD", "TN", "TX", "UT", "VT", "VA", "WA", "WV", "WI", "WY"]

# Prices 

- Data is very structured, SQL can be a good idea here

In [4]:
#API key from config file.
api_key = API_KEY

#Empty dictionary to store prices
price_df = pd.DataFrame(columns = states)

#for state in states.
for state in states:

    #query each state. ALL = average combined rates, A = Annual. --> we can get more specific and get Residential, bussiness, etc. 
    priceQuery = f"ELEC.PRICE.{state}-ALL.A"

    #Api url.
    url = f"http://api.eia.gov/series/?api_key={API_KEY}&series_id={priceQuery}"

    #Api reqeust. 
    statePriceRequest = requests.get(url)

    #Pulls list of list of with the information of prices and yaer.
    yearPricesList = statePriceRequest.json()['series'][0]['data']
    
    #Parse through the list and build DataFrame.
    for priceYear in yearPricesList:

        price_df.loc[priceYear[0],state] = priceYear[1]

#Save to Sqlite 
price_df.to_sql('Average Energy Prices', conn, if_exists='replace')

In [16]:
#To view the sqlite 
prices = pd.read_sql_query("SELECT * from 'Average Energy Prices'", conn)
prices

2020-09-04 20:31:02,751 INFO sqlalchemy.engine.base.Engine SELECT * from 'Average Energy Prices'
2020-09-04 20:31:02,752 INFO sqlalchemy.engine.base.Engine ()


,index,AL,AK,AZ,AR,CA,CO,CT,DC,DE,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,2019,9.88,20.43,10.64,8.13,17.04,10.02,18.75,12.14,10.64,...,9.93,9.75,8.8,8.32,15.34,9.53,8.06,8.4,11.04,8.12
1,2018,9.63,19.36,10.85,7.78,16.58,10.02,18.41,12.03,10.55,...,9.97,9.58,8.48,8.21,15.13,9.48,8,8.72,10.58,8.09
2,2017,9.83,19.10,10.64,8.26,16.06,9.99,17.55,11.80,10.90,...,10.05,9.45,8.38,8.60,14.60,9.18,7.94,9,10.76,8.28
3,2016,9.56,17.93,10.33,8.13,15.23,9.83,17.24,11.73,11.09,...,9.83,9.23,8.43,8.72,14.46,9.09,7.68,8.98,10.67,8.19
4,2015,9.33,17.59,10.34,8.19,15.42,9.94,17.77,12.07,11.17,...,9.47,9.30,8.7,8.54,14.41,9.31,7.4,8.11,10.73,7.97
5,2014,9.27,17.46,10.18,7.90,15.15,10.06,17.05,12.11,11.22,...,9.05,9.40,8.94,8.35,14.57,9.17,7.13,7.65,10.57,7.76
6,2013,9.02,16.49,10.14,7.93,14.30,9.88,15.66,11.85,10.90,...,8.86,9.13,8.66,8.15,14.61,8.96,7.09,7.91,10.51,7.55
7,2012,9.18,16.33,9.81,7.62,13.53,9.39,15.54,11.85,11.06,...,8.49,9.27,8.55,7.84,14.22,9.07,6.94,8.14,10.28,7.19
8,2011,9.10,16.08,9.71,7.43,13.05,9.39,16.35,12.81,11.48,...,8.05,9.28,9,7.13,13.80,8.84,6.78,7.88,10.21,6.58
9,2010,8.89,14.76,9.69,7.28,13.01,9.15,17.38,13.35,11.97,...,7.82,8.61,9.34,6.94,13.24,8.69,6.66,7.45,9.78,6.20


# Energy Generation

- Data is very unstructured, we might have to use NoSQL (Mongo_dB)


<img src="images/2019 US Energy Distribution.png">

In [63]:
#list of the US Energy Sources by EIA.
energySources = {"Natural Gas" : "NG",
                "Coal" : "COW",
                "Petroleum Liquids" : "PEL", 
                "Petroleum Coke" : "PC", 
                "Other Gasses": "OOG",
                "Nuclear" : "NUC",
                "Hydroelectric" : 'HYC',
                "Wind" : "WND",
                'Wood and Wood Derived Fuels' : 'WWW',
                'Other Biomass' : 'WAS',
                'Geothermal' : 'GEO',
                'All Other Renewables' : "AOR",
                "All Solar": 'TSN',
                'Other' : 'OTH',
                'Utility Scale Thermal' : 'STH',
                'Hydroelectic Pumped Storage' : 'HPS'
                }

#For state in states
for state in states:

    #Empty Df to store data
    energySrouce_df = pd.DataFrame(columns = energySources.keys(), index = range(2001,2020))

    
    try:
        #For every statem loop each energy source.
        for energySourceKey, energySourceValue in zip(energySources.keys(), energySources.values()):
            
            #builds api query.
            priceQuery = f"ELEC.GEN.{energySourceValue}-{state}-99.A"
            
            #API url
            url = f"http://api.eia.gov/series/?api_key={API_KEY}&series_id={priceQuery}"

            #Api reqeust. 
            energySourceRequest = requests.get(url, allow_redirects = False) 

            #Extracts the year-energy liss
            energySourceList = energySourceRequest.json()['series'][0]['data']

            for energy in energySourceList:
                energySrouce_df.loc[int(energy[0]), energySourceKey] = energy[1]

            energySrouce_df = energySrouce_df.fillna(0)

            #Save to Sqlite 
            energySrouce_df.to_sql(f'{state} Generation', conn, if_exists='replace')
    
    #if energy surce is not found.
    except KeyError as K:
        print(f'{K}. {energySourceKey} not found in {state}.')

        time.sleep(5)

print('Done!')

 (2005, 6386.79365, 41729.31343, 163.96753, 582.54246, 0.0, 9920.991, 1740.21896, 92.544, 740.34998, 0, 0, 0, 0, 0, 0, 0), (2006, 5358.4816, 40116.49023, 135.70123, 741.48433, 0.0, 12233.515, 1678.5983, 101.376, 773.8628, 0, 0, 0, 0, 0, 0, 0), (2007, 6488.75032, 40028.24955, 159.71219, 853.70149, 0.0, 12910.319, 1516.08286, 109.283, 785.07895, 0, 0, 0, 0, 0, 0, 0), (2008, 5243.58889, 41705.86048, 96.00967, 835.15012, 0.0, 12154.51, 1616.14196, 487.141, 775.04025, 0, 0, 0, 0, 0, 0, 0)  ... displaying 10 of 19 total bound parameter sets ...  (2018, 16798.65559, 33322.37949, 36.08083, 101.4305, 0.0, 10129.122, 2392.00955, 1637.658, 883.11537, 0, 0, 0, 0, 0, 0, 0), (2019, 21136.97903, 26348.94969, 46.54271, 89.68668, 0.0, 10030.305, 1992.70442, 1649.21371, 841.48595, 0, 0, 0, 0, 0, 0, 0))
2020-09-04 21:07:38,711 INFO sqlalchemy.engine.base.Engine COMMIT
2020-09-04 21:07:38,713 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2020-09-04 21:0

In [64]:
#To view the sqlite 
stateGeneration = pd.read_sql_query("SELECT * from 'CA Generation'", conn) #State Initials to display table.
stateGeneration

2020-09-04 21:07:54,791 INFO sqlalchemy.engine.base.Engine SELECT * from 'CA Generation'
2020-09-04 21:07:54,791 INFO sqlalchemy.engine.base.Engine ()


,index,Natural Gas,Coal,Petroleum Liquids,Petroleum Coke,Other Gasses,Nuclear,Hydroelectric,Wind,Wood and Wood Derived Fuels,Other Biomass,Geothermal,All Other Renewables,All Solar,Other,Utility Scale Thermal,Hydroelectic Pumped Storage
0,2001,111932.27100,2232.85100,935.24300,2119.65000,1130.49900,33219.520,25541.77600,3499.73800,3323.77700,2082.53626,12181.29400,21629.61626,0.00000,204.33774,0.00000,-349.689
1,2002,89624.04400,2327.80900,131.43400,1829.63200,1240.05300,34352.340,31140.62800,3802.64500,3957.58900,2018.75924,13073.61500,23406.98024,0.00000,398.10776,0.00000,-240.997
2,2003,91432.18100,2326.30500,396.08500,1996.16400,1759.01500,35593.789,36370.70400,3895.43100,3880.03700,2254.28079,12981.76300,23545.11779,0.00000,281.49521,531.65837,-912.313
3,2004,100222.23284,2237.80833,171.99778,2090.89968,2065.96501,30267.887,34140.92927,4305.87526,3826.90618,2161.92176,13105.30612,23970.89932,0.00000,428.56676,568.93000,-816.831
4,2005,93353.84929,2135.37482,256.69518,2287.00208,2279.58374,36154.898,39631.86657,4262.22901,3609.13942,2223.52616,13022.63920,23654.24713,0.00000,419.50825,534.70134,119.793
5,2006,105691.11621,2235.34193,272.15311,2096.02095,2022.44604,31958.621,48047.38008,4882.80100,3422.09289,2294.49548,12821.43400,23915.39536,0.00000,464.07650,492.62400,96.137
6,2007,115700.47038,2298.30558,277.88741,2056.08688,1818.10627,35792.490,27327.75089,5584.93335,3407.41556,2305.22831,12990.71099,24845.25722,0.00000,421.44725,553.78500,309.779
7,2008,119991.73722,2280.40050,162.06442,1579.52595,1684.86335,32482.351,24127.80962,5384.95489,3483.55495,2361.94599,12883.00000,24783.93682,0.00000,570.31095,664.60600,321.263
8,2009,113463.45471,2049.94689,136.61562,1406.23282,1622.84384,31763.804,27888.03633,5839.81324,3732.01555,2467.66026,12852.78300,25539.66161,0.00000,752.68744,613.35972,152.849
9,2010,107522.31347,2100.22129,53.65868,1005.63016,1694.64070,32200.757,33430.87002,6078.63200,3550.87737,2450.78271,12600.09800,25449.72104,0.00000,839.04819,653.67900,-171.265


# Energy Consumption by State

<img src="images/Energy Classification.png">


In [18]:
# Empty dataframe to store the data 
EndUseCosumption_df = pd.DataFrame(columns = states)

for state in states:

    #URL to scrape
    url = f"https://www.eia.gov/beta/states/states/{state}/overview"

    #Open url.
    browser.get(url)

    #Time to let the website load all the elements
    time.sleep(3)

    #save the html source.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #List of classes to parse 
    parseClasses = ["info-coal","info-natural-gas","info-petroleum","info-renewable",'info-electricity'] 

    for parseClass in parseClasses:

        yummySoup = soup.find_all("li", class_={parseClass})[0].text.split()

        typeEnergy = yummySoup[0]

        marketShare = yummySoup[2]

        EndUseCosumption_df.loc[typeEnergy,state] = marketShare

#Save to Sqlite 
EndUseCosumption_df.to_sql('End User Consumption', conn, if_exists='replace')

2020-09-04 20:38:38,824 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("End User Consumption")
2020-09-04 20:38:38,825 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:38:38,825 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("End User Consumption")
2020-09-04 20:38:38,825 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:38:38,828 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "End User Consumption" (
	"index" TEXT, 
	"AL" TEXT, 
	"AK" TEXT, 
	"AZ" TEXT, 
	"AR" TEXT, 
	"CA" TEXT, 
	"CO" TEXT, 
	"CT" TEXT, 
	"DC" TEXT, 
	"DE" TEXT, 
	"FL" TEXT, 
	"GA" TEXT, 
	"HI" TEXT, 
	"ID" TEXT, 
	"IL" TEXT, 
	"IN" TEXT, 
	"IA" TEXT, 
	"KS" TEXT, 
	"KY" TEXT, 
	"LA" TEXT, 
	"ME" TEXT, 
	"MD" TEXT, 
	"MA" TEXT, 
	"MI" TEXT, 
	"MN" TEXT, 
	"MS" TEXT, 
	"MO" TEXT, 
	"MT" TEXT, 
	"NE" TEXT, 
	"NV" TEXT, 
	"NH" TEXT, 
	"NJ" TEXT, 
	"NM" TEXT, 
	"NY" TEXT, 
	"NC" TEXT, 
	"ND" TEXT, 
	"OH" TEXT, 
	"OK" TEXT, 
	"OR" TEXT, 
	"PA" TEXT, 
	"RI" TEXT, 
	"SC" TEXT, 
	"SD" TEXT, 
	"TN

In [20]:
#To view the sqlite 
endUser = pd.read_sql_query("SELECT * from 'End User Consumption'", conn)
endUser

2020-09-04 20:44:35,584 INFO sqlalchemy.engine.base.Engine SELECT * from 'End User Consumption'
2020-09-04 20:44:35,585 INFO sqlalchemy.engine.base.Engine ()


,index,AL,AK,AZ,AR,CA,CO,CT,DC,DE,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,Coal,(4.3%),(1.2%),(0.7%),(0.5%),(0.5%),(0.3%),(0.0%),(0.0%),(0.0%),...,(1.0%),(2.4%),(0.1%),(1.4%),(0.0%),(2.7%),(0.1%),(4.7%),(1.5%),(7.3%)
1,Naturalgas,(23.3%),(55.8%),(10.7%),(26.2%),(24.7%),(34.2%),(25.0%),(35.0%),(28.5%),...,(27.3%),(19.8%),(24.8%),(30.2%),(10.8%),(18.0%),(16.7%),(35.3%),(31.2%),(39.9%)
2,Petroleum,(38.9%),(37.8%),(58.0%),(43.0%),(57.7%),(46.1%),(55.3%),(21.6%),(51.8%),...,(38.1%),(49.4%),(61.7%),(49.9%),(60.0%),(49.4%),(55.2%),(37.6%),(40.9%),(38.2%)
3,Renewableenergy,(11.6%),(1.6%),(3.3%),(9.3%),(3.4%),(2.3%),(2.9%),(1.7%),(1.1%),...,(19.6%),(5.2%),(0.9%),(1.5%),(14.8%),(5.2%),(7.5%),(3.1%),(8.9%),(1.3%)
4,Electricity,(21.9%),(3.5%),(27.4%),(21.0%),(13.7%),(17.0%),(16.8%),(41.6%),(18.6%),...,(14.0%),(23.2%),(12.5%),(16.9%),(14.3%),(24.7%),(20.5%),(19.2%),(17.5%),(13.3%)


In [19]:
#Empty dataframe to store the data 
ElectricPowerConsumption_df = pd.DataFrame(columns = states)

for state in states:

    #URL to scrape
    url = f"https://www.eia.gov/beta/states/states/{state}/overview"

    #Open url.
    browser.get(url)

    #Time to let the website load all the elements
    time.sleep(4)

    #save the html source.
    html = browser.page_source

    #Use bs4 to parse the html response.
    soup = bs(html, "html.parser")

    #List of classes to parse 
    parseClasses = ["info-coal","info-natural-gas","info-petroleum","info-renewable",'info-nuclear'] 

    for parseClass in parseClasses:

        #From the soup, the electricity generation is in the index 1.
        indexNum = 1

        #Nuclear energy is only for electricity generation. It only has 1 index = 0
        if parseClass == 'info-nuclear' :
            indexNum = 0

        yummySoup = soup.find_all("li", class_={parseClass})[indexNum].text.split()

        typeEnergy = yummySoup[0]

        marketShare = yummySoup[2]

        ElectricPowerConsumption_df.loc[typeEnergy,state] = marketShare

#Save to Sqlite 
ElectricPowerConsumption_df.to_sql('Electric Power Consumption', conn, if_exists='replace')


2020-09-04 20:44:35,314 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("Electric Power Consumption")
2020-09-04 20:44:35,314 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:44:35,315 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("Electric Power Consumption")
2020-09-04 20:44:35,315 INFO sqlalchemy.engine.base.Engine ()
2020-09-04 20:44:35,319 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE "Electric Power Consumption" (
	"index" TEXT, 
	"AL" TEXT, 
	"AK" TEXT, 
	"AZ" TEXT, 
	"AR" TEXT, 
	"CA" TEXT, 
	"CO" TEXT, 
	"CT" TEXT, 
	"DC" TEXT, 
	"DE" TEXT, 
	"FL" TEXT, 
	"GA" TEXT, 
	"HI" TEXT, 
	"ID" TEXT, 
	"IL" TEXT, 
	"IN" TEXT, 
	"IA" TEXT, 
	"KS" TEXT, 
	"KY" TEXT, 
	"LA" TEXT, 
	"ME" TEXT, 
	"MD" TEXT, 
	"MA" TEXT, 
	"MI" TEXT, 
	"MN" TEXT, 
	"MS" TEXT, 
	"MO" TEXT, 
	"MT" TEXT, 
	"NE" TEXT, 
	"NV" TEXT, 
	"NH" TEXT, 
	"NJ" TEXT, 
	"NM" TEXT, 
	"NY" TEXT, 
	"NC" TEXT, 
	"ND" TEXT, 
	"OH" TEXT, 
	"OK" TEXT, 
	"OR" TEXT, 
	"PA" TEXT, 
	"RI" TEXT, 
	"SC" TEXT, 

In [22]:
#To view the sqlite 
electricityGeneration = pd.read_sql_query("SELECT * from 'Electric Power Consumption'", conn)
electricityGeneration

2020-09-04 20:44:49,303 INFO sqlalchemy.engine.base.Engine SELECT * from 'Electric Power Consumption'
2020-09-04 20:44:49,304 INFO sqlalchemy.engine.base.Engine ()


,index,AL,AK,AZ,AR,CA,CO,CT,DC,DE,...,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY
0,Coal,(24.7%),(18.7%),(30.5%),(48.3%),(0.0%),(52.3%),(1.2%),(0.0%),(9.6%),...,(21.0%),(27.1%),(29.2%),(71.7%),(0.0%),(12.4%),(5.6%),(94.1%),(54.2%),(89.8%)
1,Naturalgas,(34.6%),(45.6%),(27.8%),(25.1%),(39.4%),(25.8%),(40.8%),(0.0%),(83.9%),...,(8.4%),(13.0%),(41.9%),(17.1%),(0.0%),(43.8%),(7.9%),(1.7%),(20.9%),(0.4%)
2,Petroleum,(0.1%),(8.7%),(0.1%),(0.1%),(0.0%),(0.0%),(0.9%),(0.0%),(4.4%),...,(0.1%),(0.2%),(0.0%),(0.1%),(0.2%),(1.1%),(0.0%),(0.2%),(0.3%),(0.1%)
3,Renewableenergy,(8.4%),(26.9%),(11.1%),(5.2%),(48.7%),(21.8%),(5.5%),(0.0%),(2.1%),...,(70.6%),(12.1%),(18.2%),(11.1%),(99.8%),(6.9%),(76.9%),(4.0%),(7.8%),(9.7%)
4,Nuclearpower,(32.2%),(0.0%),(30.5%),(21.4%),(11.8%),(0.0%),(51.5%),(0.0%),(0.0%),...,(0.0%),(47.6%),(10.7%),(0.0%),(0.0%),(35.8%),(9.6%),(0.0%),(16.8%),(0.0%)
